# TM10007 Assignment template

In [19]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

In [0]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

# Classifiers
from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from sklearn import svm
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.utils.fixes import loguniform
from sklearn.neighbors import KNeighborsClassifier


from sklearn.linear_model import Lasso, RidgeClassifier
from sklearn.feature_selection import SelectFromModel

# Data loading and cleaning

Some functions

In [0]:
# Some functions we will use
from sklearn.decomposition import PCA

def colorplot(clf, ax, x, y, h=100):
    '''
    Overlay the decision areas as colors in an axes.
    
    Input:
        clf: trained classifier
        ax: axis to overlay color mesh on
        x: feature on x-axis
        y: feature on y-axis
        h(optional): steps in the mesh
    '''
    # Create a meshgrid the size of the axis
    xstep = (x.max() - x.min() ) / 20.0
    ystep = (y.max() - y.min() ) / 20.0
    x_min, x_max = x.min() - xstep, x.max() + xstep
    y_min, y_max = y.min() - ystep, y.max() + ystep
    h = max((x_max - x_min, y_max - y_min))/h
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    if hasattr(clf, "decision_function"):
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    if len(Z.shape) > 1:
        Z = Z[:, 1]
    
    # Put the result into a color plot
    cm = plt.cm.RdBu_r
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)
    del xx, yy, x_min, x_max, y_min, y_max, Z, cm



Data loading and handling missing data. 

In [22]:
# Data loading functions. 
from adni.load_data import load_data 

data = load_data()
print(data.head())
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
#data_missing = data.isnull()
#print(data_missing.index[data_missing == True])

# Creating vector of labels
data_no_label = data.drop(columns='label')
print(f'The number of features: {len(data_no_label.columns)}')
x = data_no_label
y = data['label']

# Split data
X_train, X_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2)

def removezeros(X):
   X_nozeros = X.loc[:, (X != 0).any(axis=0)]
   return X_nozeros

X_train_df = removezeros(X_train)
X_test_df = removezeros(X_test)
print(f'The number of samples: {len(X_train_df.index)}')
print(f'The number of columns: {len(X_train_df.columns)}')
X_train = X_train_df.values
X_test = X_test_df.values

x_nan = np.isnan(X_train)
print(f'The number of found NaN is: {np.sum(x_nan)}')
#x_zero = np.where(X_train == 0)[0]
#print(f'The number of found zeros is: {len(x_zero)}')

y_train_bin = preprocessing.label_binarize(y_train,['AD','CN'])
y_train_bin = [i[0] for i in y_train_bin]

print(y_train_bin)

                     hf_energy  ...  vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0
ID                              ...                                         
0_002_S_0413_bl_0  5403.229408  ...                             4.207831e-10
0_002_S_0559_bl_0   922.459760  ...                             3.137258e-10
0_002_S_0619_bl_0  1674.765276  ...                             4.219847e-10
0_002_S_0685_bl_0  3443.797200  ...                             5.954654e-11
0_002_S_0816_bl_0   914.469846  ...                             4.174750e-10

[5 rows x 268 columns]
The number of samples: 855
The number of columns: 268
The number of features: 267
The number of samples: 684
The number of columns: 263
The number of found NaN is: 0
[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 

# Feature scaling and selection


In [23]:
# Scaling of features
scaler = preprocessing.RobustScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# Feature selection
selector = SelectFromModel(estimator=Lasso(alpha=10**(-10), random_state=42), threshold='median')
selector.fit(X_train_scaled, y_train_bin)
n_original = X_train_scaled.shape[1]
X_train_test = selector.transform(X_train_scaled)
n_selected = X_train_test.shape[1]
print(f"Selected {n_selected} from {n_original} features.")
"""
# Create the RFE object and compute a cross-validated score.
svc = SVC(kernel="linear")
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
              scoring='roc_auc')
select = rfecv.fit(X_train_scaled, y_train)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

print("Optimal number of features : %d" % rfecv.n_features_)
"""

Selected 132 from 263 features.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.24538953137276, tolerance: 0.016383625730994143
  positive)


'\n# Create the RFE object and compute a cross-validated score.\nsvc = SVC(kernel="linear")\n# The "accuracy" scoring is proportional to the number of correct\n# classifications\nrfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),\n              scoring=\'roc_auc\')\nselect = rfecv.fit(X_train_scaled, y_train)\n\n# Plot number of features VS. cross-validation scores\nplt.figure()\nplt.xlabel("Number of features selected")\nplt.ylabel("Cross validation score (nb of correct classifications)")\nplt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)\nplt.show()\n\nprint("Optimal number of features : %d" % rfecv.n_features_)\n'

In [24]:
"""
# Get new X_train with only important features 
features = X_train_df * select.support_
x_train_df = removezeros(features)
x_train = x_train_df.values
"""

'\n# Get new X_train with only important features \nfeatures = X_train_df * select.support_\nx_train_df = removezeros(features)\nx_train = x_train_df.values\n'

In [25]:
 # Train Classifier  
# defining parameter range 
param_grid = {'C': loguniform(1e0, 1e3), 
              'gamma': loguniform(1e-4, 1e0), 
              'kernel': ['rbf','poly', 'linear']}  
  
grid = RandomizedSearchCV(SVC(), param_grid, n_iter=10, refit = True, verbose = 3, random_state=42) 
  
# fitting the model for grid search 
grid.fit(x_train, y_train)

print("The best classifier is: ", grid.best_estimator_)



NameError: ignored